## 測試model性能(適應能力)

In [1]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable

from NIN_v1_model import NIN_v1  # 載入我的model
from NIN_v2_model import NIN_v2
from InceptionV3_model import Inception

In [2]:
# test_function
def test(testloader, model, use_GPU, loss_func):
    model.eval()
    val_loss = 0
    val_correct_num = 0
    with torch.no_grad():
        for i, (data, target) in enumerate(testloader):
            if use_GPU == True:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            
            # 算acc方法1:
            print("正確class = ",target.data)
            val_loss += loss_func(output, target).item()  # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
            print("預測class = ",pred)
            print("預測output = ",output,"\n")
            val_correct_num += pred.eq(target.data.view_as(pred)).cpu().sum()
            # 算acc方法2:
            #_, idx = torch.max(output.data, dim=1)
            #val_correct_num += torch.sum(idx == target)

        val_loss /= len(testloader)


        val_correct_rate = 100. * val_correct_num / len(testloader.dataset)
        result = 'Val set -> Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)'.format(
            val_loss, val_correct_num, len(testloader.dataset), val_correct_rate) 
        print(result)
    return val_correct_rate, val_loss

In [3]:
# 參數設定
use_GPU = True
loss_func = torch.nn.CrossEntropyLoss() 
img_size = 224

# 資料前處理
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
transform_test = transforms.Compose([
    transforms.Resize((img_size,img_size)), #長=256 寬=等比例縮小
    #transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    normalize
])


test_data = datasets.ImageFolder('test_data/', transform=transform_test)
testloader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
print("test_data資料夾，圖片有", len(testloader.dataset), "張\n")

# 加載訓練好的模型
best_model = torch.load('best_model.pth')
    
test_acc, test_loss = test(testloader, best_model, use_GPU, loss_func)

test_data資料夾，圖片有 6 張

正確class =  tensor([0], device='cuda:0')
預測class =  tensor([[2]], device='cuda:0')
預測output =  tensor([[-0.2875, -0.4370,  0.7049]], device='cuda:0') 

正確class =  tensor([0], device='cuda:0')
預測class =  tensor([[2]], device='cuda:0')
預測output =  tensor([[-0.6164, -0.9542,  1.6220]], device='cuda:0') 

正確class =  tensor([1], device='cuda:0')
預測class =  tensor([[1]], device='cuda:0')
預測output =  tensor([[-0.0192,  0.6983, -0.7980]], device='cuda:0') 

正確class =  tensor([1], device='cuda:0')
預測class =  tensor([[2]], device='cuda:0')
預測output =  tensor([[-0.7337, -0.2566,  1.0025]], device='cuda:0') 

正確class =  tensor([2], device='cuda:0')
預測class =  tensor([[2]], device='cuda:0')
預測output =  tensor([[-0.8271, -0.3096,  1.1503]], device='cuda:0') 

正確class =  tensor([2], device='cuda:0')
預測class =  tensor([[2]], device='cuda:0')
預測output =  tensor([[-1.0314, -0.6792,  1.7354]], device='cuda:0') 

Val set -> Average loss: 1.0926, Accuracy: 3/6 (50.000%)
